In [1]:
#Data Download directory

dir="/home/advaita/Downloads/data"

In [2]:
#Imports
import numpy as np 
import pandas as pd 

import seaborn as sns
from matplotlib import pyplot as plt
sns.set_style("whitegrid")
%matplotlib inline

import warnings
warnings.filterwarnings("ignore")
import os 

In [3]:
#Frames


df_stops=pd.read_csv(dir+"/pit_stops.csv")
df_constructor_standing=pd.read_csv(dir+"/constructor_standings.csv")
df_races=pd.read_csv(dir+"/races.csv")

In [4]:
#Merge Frames

commmon_columns_CS_PT=["raceId"]
commmon_columns_CS_PT_RC=["raceId"]

merged_CS_PT=pd.merge(df_stops,df_constructor_standing,on=commmon_columns_CS_PT,how="outer")
merged_CS_PT_RC=pd.merge(merged_CS_PT,df_races,on=commmon_columns_CS_PT_RC,how="outer")



In [ ]:
print(merged_CS_PT_RC.describe())

In [17]:
#Data cleaning


# nan_count = merged_CS_PT_RC.isna().sum()
# print(nan_count)

# rows_with_nan = merged_CS_PT_RC[merged_CS_PT_RC.isna().any(axis=1)]
# print(rows_with_nan)

df_cleaned=merged_CS_PT_RC.drop(columns=["round","date","time_y","url","fp1_date","fp1_time","fp3_time","quali_date","fp2_date","fp2_time","fp3_date","quali_time","sprint_date","sprint_time"])
# nan_count = merged_CS_PT_RC.isna().sum()
# print(nan_count)


# Assuming df is your DataFrame

# Print all rows containing NaN values
df_cleaned.dropna(inplace=True)
df_sampled=df_cleaned.sample(n=1000,random_state=42)

# print(df_cleaned.describe())
print(df_cleaned.describe())

              raceId       driverId           stop            lap  \
count  116628.000000  116628.000000  116628.000000  116628.000000   
mean      970.411754     532.056590       1.799705      25.332390   
std        89.497698     388.151594       1.510880      14.841635   
min       841.000000       1.000000       1.000000       1.000000   
25%       887.000000      18.000000       1.000000      13.000000   
50%       960.000000     816.000000       2.000000      25.000000   
75%      1052.000000     832.000000       2.000000      36.000000   
max      1132.000000     860.000000      70.000000      78.000000   

       milliseconds  constructorStandingsId  constructorId         points  \
count  1.166280e+05           116628.000000  116628.000000  116628.000000   
mean   8.283363e+04            26588.007408      71.625339      99.149823   
std    3.049661e+05             1331.130241      86.192215     139.625558   
min    1.289700e+04            24540.000000       1.000000       0.000

Visuilazing data

In [ ]:
sns.pairplot(df_sampled)

In [ ]:
for i in df_cleaned:
    plt.figure(figsize=(10,5))
    sns.violinplot(data=df_sampled,y=i)

In [20]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

net=nn.Sequential(
    nn.Linear(3,30),
    nn.ELU(),
    nn.Linear(30,40),
    nn.ELU(),
    nn.Linear(40,1)
)

optimizer = optim.Adam(net.parameters(), lr=0.001)  
criterion=nn.MSELoss()

input_data = df_cleaned[['circuitId', 'driverId', 'position']].values
target_data = df_cleaned['points'].values

X_train, X_test, y_train, y_test = train_test_split(input_data, target_data, test_size=0.3, random_state=42)

X_train_tensor = torch.tensor(X_train, dtype=torch.float32).to(device)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).view(-1, 1)  .to(device)

X_test_tensor = torch.tensor(X_test, dtype=torch.float32).to(device)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32).view(-1, 1).to(device)


net.to(device)

for epoch in range(10000000):
    optimizer.zero_grad()

    pred=net(X_train_tensor)
    loss=criterion(pred,y_train_tensor)
    
    loss.backward()
    
    optimizer.step()
    
    if (epoch+1) % 10 == 0:
        print(f'Loss: {loss.item()}')
        
net.eval()
with torch.no_grad(): 
    y_test_pred = net(X_test_tensor)
    test_loss = criterion(y_test_pred, y_test_tensor)
    print(f'\nTest Loss: {test_loss.item()}')
    
    



Loss: 27016.822265625
Loss: 26513.611328125
Loss: 26034.916015625
Loss: 25531.2421875
Loss: 25004.484375
Loss: 24468.484375
Loss: 23943.09765625
Loss: 23452.501953125
Loss: 23021.33984375
Loss: 22670.45703125
Loss: 22410.501953125
Loss: 22238.47265625
Loss: 22137.9921875
Loss: 22085.35546875
Loss: 22058.47265625
Loss: 22042.279296875
Loss: 22029.1328125
Loss: 22016.263671875
Loss: 22001.029296875
Loss: 21986.701171875
Loss: 21972.498046875
Loss: 21957.833984375
Loss: 21943.21875
Loss: 21928.318359375
Loss: 21913.119140625
Loss: 21897.55859375
Loss: 21881.603515625
Loss: 21865.228515625
Loss: 21848.416015625
Loss: 21831.134765625
Loss: 21813.353515625
Loss: 21795.03515625
Loss: 21776.142578125
Loss: 21756.669921875
Loss: 21736.666015625
Loss: 21716.16796875
Loss: 21695.1796875
Loss: 21673.6875
Loss: 21651.67578125
Loss: 21629.12109375
Loss: 21606.005859375
Loss: 21582.3046875
Loss: 21558.00390625
Loss: 21533.0703125
Loss: 21507.486328125
Loss: 21481.224609375
Loss: 21454.259765625
Loss:

KeyboardInterrupt: 

raceId       driverId           stop            lap  \
count  116628.000000  116628.000000  116628.000000  116628.000000   
mean      970.411754     532.056590       1.799705      25.332390   
std        89.497698     388.151594       1.510880      14.841635   
min       841.000000       1.000000       1.000000       1.000000   
25%       887.000000      18.000000       1.000000      13.000000   
50%       960.000000     816.000000       2.000000      25.000000   
75%      1052.000000     832.000000       2.000000      36.000000   
max      1132.000000     860.000000      70.000000      78.000000   

       milliseconds  constructorStandingsId  constructorId         points  \
count  1.166280e+05           116628.000000  116628.000000  116628.000000   
mean   8.283363e+04            26588.007408      71.625339      99.149823   
std    3.049661e+05             1331.130241      86.192215     139.625558   
min    1.289700e+04            24540.000000       1.000000       0.000000   
25%    2.190300e+04            25213.000000       5.000000       6.000000   
50%    2.357200e+04            26600.000000      10.000000      40.000000   
75%    2.637600e+04            27703.000000     131.000000     130.000000   
max    3.069017e+06            28852.000000     215.000000     860.000000   

            position           wins           year      circuitId  
count  116628.000000  116628.000000  116628.000000  116628.000000  
mean        5.834354       0.967323    2016.686953      22.638500  
std         3.100320       2.617325       4.138606      23.394307  
min         1.000000       0.000000    2011.000000       1.000000  
25%         3.000000       0.000000    2013.000000       7.000000  
50%         6.000000       0.000000    2016.000000      14.000000  
75%         8.000000       0.000000    2021.000000      24.000000  
max        12.000000      21.000000    2024.000000      80.000000  